# 💘 The Stolen Valentine Budget

<img src="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/stolen_valentine_banner.png" width="60%">

## 🕵️ A Corporate SQL Investigation - Introduction

Lunaris Systems approved a modest Valentine’s Day budget for its **Madrid office**.

The plan was simple: **a small internal celebration** to thank employees for their work.


The budget was approved. Expense reports were submitted. Everything appeared routine…

Until Finance reviewed the February numbers.

⚠️ The Valentine budget appears to have been quietly drained.

Several expense reports contain purchases that raise questions.  
Audit suspects that one employee may have disguised personal spending as legitimate office expenses.


#### 🎯 Your mission

> Use SQL to investigate the data and identify the prime suspect.

This will not be a step-by-step recipe.

You will need to:

- Explore the company’s expense data
- Test investigative hypotheses
- Distinguish real clues from misleading ones
- Apply precise filtering and aggregation
- Produce a final, defensible SQL report

Your final submission must clearly identify the employee you believe is responsible, and the SQL logic that justifies that conclusion.

Be precise. Be rigorous. Finance will audit your audit.

## **SQL Environment Setup (do not edit)**

In [19]:
# @title
%%capture
!mkdir -p notebook_lib
!wget -q -O notebook_lib/sql_runner.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/8021f5c05b7d973b8db549a1398a3c9a5c7829d5/notebook_lib/sql_runner.py
!wget -q -O notebook_lib/validators.py \
  https://raw.githubusercontent.com/Haross/sql_notebook/7baff2c6485cdf641cabcdb55d92a51317cd18b9/notebook_lib/validators.py

!wget -q -O data.db \
  https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/data_A.db

from notebook_lib.sql_runner import make_sql_runner
from notebook_lib.validators import make_df_validator_nospoilers, check_process_rules

import sqlite3
import pandas as pd
from pathlib import Path


In [20]:
# @title
DB_FILE = 'data.db'

conn = sqlite3.connect(DB_FILE)
print(f"Database ready ✅ ({DB_FILE})")


Database ready ✅ (data.db)


## 🗂️ The Database

The investigation involves five tables:

1. **employees**: Contains information about Lunaris employees.
2. **vendors**: Businesses where purchases were made.
3. **expense_reports**: Expense reports submitted by employees.
4. **expenses**: Individual expense line items inside each report.
5. **valentine_budget**: The officially approved Valentine budget for February (by office).

No single table contains the full story.

To solve the case, you will need to:
- Follow the money across multiple tables
- Understand how reports connect to employees
- Connect purchases to vendors
- Compare spending against the approved budget

Careful joins will matter.

In [21]:
# @title ER Diagram
%%html
<img id="er-img" style="width:75%; max-width:100%; height:auto;"
     data-light="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/ER_stolen_valentine_budget.png"
     data-dark="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/ER_stolen_valentine_budget_black.png"
     alt="ER diagram">

<script>
  const img = document.getElementById("er-img");

  function isDarkTheme() {
    // Colab sets html[theme=dark] on the top document
    const themeAttr = document.documentElement.getAttribute("theme");
    if (themeAttr) return themeAttr === "dark";

    // fallback: OS/browser preference
    return window.matchMedia && window.matchMedia("(prefers-color-scheme: dark)").matches;
  }

  function updateImage() {
    img.src = isDarkTheme() ? img.dataset.dark : img.dataset.light;
  }

  updateImage();

  // React to Colab theme toggles (attribute changes)
  new MutationObserver(updateImage).observe(document.documentElement, {
    attributes: true,
    attributeFilter: ["theme"]
  });

  // React to OS/browser theme changes (fallback)
  if (window.matchMedia) {
    const mq = window.matchMedia("(prefers-color-scheme: dark)");
    mq.addEventListener?.("change", updateImage);
    mq.addListener?.(updateImage); // older browsers
  }
</script>

## ⚖️ Investigation Rules

To keep the investigation fair, you may use **only the SQL concepts covered in class so far**:

- `SELECT`, `FROM`, `WHERE`
- `DISTINCT`
- `AND`, `OR`
- `ORDER BY`
- `JOIN ... ON`
- Aggregate functions: `COUNT`, `SUM`, `AVG`, `MIN`, `MAX`
- `GROUP BY`
- `HAVING`

You may **not** use any other advanced SQL features not covered in class such as subqueries, window functions, etc.

This case is designed to be solved using **clean relational reasoning**, not advanced syntax.

If your logic is correct, you do not need more tools.

> Overly complex queries will not receive additional credit.  
> Clarity and correctness matter more than cleverness.



## 🎯 The Goal of This Investigation

Your task is to determine which employee is responsible for the suspicious Valentine spending.

By the end of this notebook, you will submit one final SQL query that supports your conclusion.

Assume Finance will review your report.

Your conclusion must be supported by clear evidence, not assumptions.

## 📘 How This Investigation Is Structured

This notebook is divided into three sections:

### Section 1 — Evidence Room

You will explore the database and understand how the tables relate to each other.

Queries are exploratory and intended to help you understand the data.  
While optional, they are strongly recommended.

### Section 2 — Evaluating the Claims

In this section, you will examine investigative claims and analyze patterns in the data.

Some sections will be marked:

> 📝 Submit Query

These require you to submit the SQL query itself.

Other prompts are exploratory and are designed to guide your reasoning toward the final investigation.

### Section 3 — Discover the Prime Suspect

You will write one final SQL query that identifies the prime suspect.

This is the most important part of the assignment.

Your submission must include:
- The final SQL query
- The name of the employee you identified
- A brief explanation of your reasoning

Assume Finance will challenge your conclusion.
Your explanation must withstand scrutiny.


# 🔎 Section 1 — The Evidence Room

<img src="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/evidence_room_banner.png" width="50%">

In this section, your objective is simple:

> Understand the structure of the data and what information is available.

Resist the temptation to jump to conclusions.

Do **not** try to identify the suspect yet.

First, learn how the expense system works:

- How employees are assigned to offices  
- How expense reports are submitted and tracked  
- How individual expenses are recorded  
- How vendors are categorized  
- How budgets are defined  

Strong investigations begin with structural understanding.

In [43]:
# @title 1.1 - The Valentine Budget

make_sql_runner(
    conn,
    runner_id="ex1_1",
    description_md="""
## 🧾 1.1 The Valentine Budget

Start with the official budget table.

Explore **valentine_budget** to understand:

- Which offices have Valentine budgets
- How budgets vary by office
- How the month is represented in the data

As you explore, make sure you can locate the Valentine budget for **February 2026** for any office.

You will use this table later to evaluate whether spending appears normal or excessive.


""",
    )



In [45]:
# @title 1.2 - Who Works at Lunaris?

make_sql_runner(
    conn,
    runner_id="ex1_2",
    description_md="""
## 👥 1.2. Who Works at Lunaris?

Before analyzing spending, understand the workforce.

Explore the **employees** table.

As you investigate, consider:

- How many distinct offices are represented?
- Are all employees located in the same office?
- Which column determines an employee’s office assignment?
- Why might office location matter in a budget investigation?

Avoid conclusions, focus on mapping the organization.

""",
    )



In [46]:
# @title 1.3 - Where Are Purchases Made?

make_sql_runner(
    conn,
    runner_id="ex1_3",
    description_md="""
## 🏢 1.3 Where Are Purchases Made?

Explore the **vendors** table to understand:

- What vendors exist
- How vendors are categorized (**vendor_type**)
- What kinds of purchases are possible

As you explore, consider:

- Which vendor types look typical for office operations?
- Which vendor types might raise questions in a Valentine-budget context?

Do not draw conclusions yet, just map the landscape of possible spending.

""",
    )



In [47]:
# @title 1.4 - How Do Expense Reports Work?

make_sql_runner(
    conn,
    runner_id="ex1_4",
    description_md="""
## 🧾 1.4 How Do Expense Reports Work?

Expense reports group multiple expense line items.

Explore the **expense_reports** table carefully.

As you investigate, consider:

- What status values exist?
- How common is each status?
- Do all statuses appear to contain expenses (once joined to **expenses**)?
- What might status imply for a financial investigation?

Understanding report status will matter later.

""",
    )



In [48]:
# @title 1.5 - What Do Individual Expenses Look Like?

make_sql_runner(
    conn,
    runner_id="ex1_5",
    description_md="""
## 💳 1.5 What Do Individual Expenses Look Like?

The real details live in the **expenses** table.

Explore it carefully. Pay attention to:

- The different expense categories
- The date range of recorded expenses (`expense_date`)
- Whether any receipts are missing (`receipt_id` IS NULL)

Notice that the database contains multiple date fields across tables.
Later, you will need to decide which date best represents when spending occurred.
""",
    )



## End of Section 1  — Evidence Review

At this stage, you should clearly understand:

- How the five tables relate to each other  
- What information each table contains  
- Which columns may be relevant for scoping and filtering  
- What time periods appear in the data  

You are **not** solving the case yet.

You are building structural understanding.

Strong investigations begin with context, not conclusions.

# 🧠 Section 2 — Testing the Hypotheses

<img src="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/hypothesis_banner.png" width="50%">

Finance and Audit have proposed several possible explanations for the budget irregularities.

Not all of them are reliable.

Your task in this section is to **evaluate each claim using SQL**.

Some explanations may sound convincing at first glance.  
Others may appear harmless, until examined closely.

As investigators, you must rely on evidence, not intuition.

Every conclusion must be supported by data.

In [34]:
# @title Claim 1 — Only Approved Reports Affect the Budget

make_sql_runner(
    conn,
    runner_id="claim_1",
    description_md="""
## 🧾 Claim 1: Only Approved Reports Affect the Budget

Audit suggests that only **approved expense reports** actually impacted Lunaris’ budget.

Before adopting that rule, examine how the reporting system is actually being used.

### 🔎 Suggested exploration

Use SQL to investigate:

- What status values exist in the system, and how common each one is
- Whether non-approved reports still contain recorded expenses
- Whether money is concentrated in one status or spread across several

As you analyze, consider:

- Do expenses exist under every status?
- Is money concentrated in one status category?
- If you were building a financial report, which statuses would you reasonably include or exclude, and why?

For now, do not choose a final rule.
Your job is to document what the data suggests and what assumptions would be reasonable.

""",
    )



In [35]:
# @title Claim 2 — It All Happened on Valentine's Day

make_sql_runner(
    conn,
    runner_id="claim_2",
    description_md="""
## 💘 Claim 2: “It All Happened on Valentine’s Day”

A manager insists the budget was drained **on February 14**, because “that’s the only day that matters.”

Before accepting that story, verify whether spending is:

- concentrated on a single day, or
- spread across multiple days throughout February.

Your job here is to produce evidence that either supports or weakens the February 14 claim.

As you explore, think about:

- Which date column represents when money was actually spent
- Whether spikes are driven by **many transactions** or **a few expensive ones**
- Whether patterns look similar across offices, or if one office behaves differently

Avoid locking onto February 14 too early.
Start broad, then narrow only if the data forces you to.
""",
    )



In [36]:
# @title 📝 Submit Query A - Finance Dashboard Request

make_sql_runner(
    conn,
    runner_id="hypothesis_submit_2",
    description_md="""
## 📊 Finance Request — February Office Spending Intensity

The CFO wants a high-level overview of how each office behaved financially during February 2026.

She is not only interested in total spending.
She wants to understand **how intensely each office spent money when it was actually active**.

In particular, she wants to compare offices in terms of:

- Overall February spending
- The number of transactions recorded
- The average amount spent on the days when expenses were recorded

An office that spent large amounts concentrated on only a few days may indicate unusual behavior compared to one with steady activity throughout the month.

Prepare a report that allows Finance to compare offices side by side for February 2026 and identify potential outliers.

Think carefully about:

- Which date represents when money was actually spent
- How to define the number of days with recorded activity
- Which measures help distinguish steady activity from concentrated bursts

Write one SQL query that produces this report.
""",
    )



In [40]:
# @title Claim 3 — The Misuse is Hidden in a Category

make_sql_runner(
    conn,
    runner_id="hypothesis_3",
    description_md="""
## 🌹 Claim 3 — The Misuse is Hidden in a Category

Audit believes the spending misuse may be disguised under a legitimate-looking **expense category**, but they don’t know which one.

Your job is not to accuse an employee yet.
Your job is to identify **which categories deserve deeper scrutiny** and explain why.

When you explore, make sure your analysis uses the same scope you believe is appropriate for the Valentine budget review. (you will need to justify that scope later).

As you investigate, try to answer questions like:

- Which categories drive the largest portion of spending in the relevant scope?
- Are any categories both **frequent** and **financially significant**?
- Which categories look operational (expected), and which look harder to justify under a Valentine budget?

>⚠️ Audit reminder: Small routine expenses can be common in corporate data and may inflate counts without being misuse.
>Do not treat “many small charges” the same as “few high-cost purchases” unless you can defend it.

After exploring, think about:

- Which category (or categories) raise suspicion and warrant further investigation
- What specific pattern made you choose them (frequency, total money, vendor type, etc.)?
""",
    )



In [41]:
# @title Claim 4 — Missing Receipts May Indicate Suspicion

make_sql_runner(
    conn,
    runner_id="claim_4",
    description_md="""
## 🧾 Claim 4: Missing Receipts May Indicate Suspicion

Audit flagged that some expenses appear to be missing receipt documentation.

Missing receipts can reflect harmless process issues (lost paperwork), or it can signal higher-risk spending.
Your job is to evaluate whether missing receipts look **random**, or **concentrated** in specific parts of the data.

### 🔎 Suggested exploration

Use SQL to investigate:

- How common missing receipts are overall
- Whether missing receipts are concentrated around certain vendors, categories, or offices
- Whether the missing-receipt amounts tend to be small routine purchases or higher-cost transactions

Keep the Valentine-budget investigation in mind, but do not assume missing receipts automatically imply misuse.

Avoid jumping to conclusions.
Look for patterns.

""",
    )



In [42]:
# @title 📝 Submit Query B — Compliance Documentation Risk

make_sql_runner(
    conn,
    runner_id="submit_query_b",
    description_md="""
## 🧾 Compliance Memo — Documentation Risk Review

**From:** Head of Compliance
**To:** Internal Audit Team

As part of our February 2026 internal controls review, we are examining receipt documentation practices across all offices.

Preliminary checks suggest that some expenses were recorded without proper receipt information.
However, it is unclear whether this issue is random or concentrated around specific vendors or offices.

Prepare a report that allows Compliance to identify:

- Which office–vendor combinations are most frequently associated with missing receipts
- How much money is involved in those undocumented expenses

Focus only on transactions that occurred in February 2026 (use the expense date).

The report should clearly highlight both how often receipts are missing and the total monetary impact.

Write one SQL query that produces this report.
""",
    )



## End of Section 2 — Evidence Synthesis

At this stage, you should have:

- Examined how report statuses affect financial interpretation
- Tested whether February 14 alone explains the spending pattern
- Identified which categories dominate spending
- Investigated whether missing receipts cluster around specific vendors

Now comes the critical step:

> Which filters are logically necessary to identify the true suspect?

Before proceeding to the next part, reflect on your **scope rules** (you will defend these in Section 3):

1) Which date defines “February spending” in your analysis, and why?  
2) Which report statuses you will include/exclude, and why?  
3) Which office(s) you will focus on, and why?

The final investigation will require precision.

Over-filtering may hide the suspect.  
Under-filtering may implicate the wrong employee.

Proceed carefully, your conclusion must be defensible.


# ⚖️ Section 3 — The Final Case Report

<img src="https://raw.githubusercontent.com/Haross/sql_notebook/main/assignments/The_Stolen_Valentine_Budget/final_case_banner.png" width="50%">

You have evaluated the claims.  
You have examined the evidence.

You have analyzed:

- Report statuses  
- Office and date spending patterns  
- Category-level spending  
- Receipt irregularities  

Now Audit requires a formal conclusion.

This is no longer exploration.

It is your responsibility to present a clear, defensible finding.


## Audit Request

Audit requires a clear and defensible SQL report identifying the prime suspect in the Valentine budget case.

Your task:

> Write **one** SQL query that identifies the employee whose spending patterns in February 2026 indicate misuse of the Valentine budget.

You must determine:

- Which filters are logically necessary  
- Which clues were misleading  
- Which expense categories are relevant  
- Which records should be excluded  
- Which aggregation rules meaningfully identify suspicious behavior  

Your query must rely only on data-driven logic.  

Do **not** hard-code employee names/IDs, and do not choose filters **only because** they produce a particular output.

If you use thresholds (for example, a minimum amount or a minimum number of suspicious expenses), you must justify them.  

If changing your threshold slightly would identify a different employee, you must explain why your chosen threshold is still appropriate.

---

### 📝 Submission Requirements (Canvas)

You must submit:

1. Your final SQL query  
2. The employee(s) identified by your query  
3. A short justification explaining:
   - Why your filters are logically necessary  
   - Why alternative filters would be incorrect  
   - Why your conclusion is defensible  

If your query returns multiple employees, you must explain why.  
If it returns none, you must explain why.  

Your grade will be based on the strength of your reasoning.


In [33]:
# @title 3.1 Discover the Prime Suspect

make_sql_runner(
    conn,
    runner_id="3_1",
    description_md="""
## 🔍 3.1 Discover the Prime Suspect

You will now produce the final audit report.

Your final query must return **exactly** these columns:

- `employee_id`
- `first_name`
- `last_name`
- `suspicious_expenses_count`
- `total_suspicious_amount`
- `distinct_vendors`
- `avg_expense_amount`

Sort the results so the most suspicious employee appears first.

In a strong audit report, one employee should clearly stand out.
If your query returns multiple employees (or none), your scope rules or logic need to be explained.

Choose your filters carefully and make sure you can defend every decision.

""",
    )

